In [20]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime, timedelta
from config import password, login, server
import time
import pytz

In [2]:
print(login, password)

5916003 IEQHeeen


In [17]:
mt5.initialize(login=login, server=server, password=password)

True

In [50]:
# settings
symbol = 'USDJPY'
timeframe = mt5.TIMEFRAME_M5
start_pos = 0
num_bars = 150
volume = 1

fsma_period = 14
ssma_period = 28
timezone = pytz.timezone('America/Sao_Paulo')
date_from = datetime(2023, 10, 9, tzinfo=timezone)
date_to = datetime(2023, 9, 10, tzinfo=timezone)
# Obtendo a data e hora atuais
current_day = datetime.now(tz=timezone).replace(microsecond=0)
one_year_before = current_day - timedelta(days=365)
print(date_from)
print(date_to)
print(current_day)
print(one_year_before)

2023-10-09 00:00:00-03:06
2023-09-10 00:00:00-03:06
2023-10-11 01:07:44-03:00
2022-10-11 01:07:44-03:00


In [22]:
# display EURJPY symbol properties
symbol_info=mt5.symbol_info("WDO$")
print(date_from)

2023-10-09 00:00:00-03:06


In [ ]:
df = pd.DataFrame(bars)[['time', 'open', 'close', 'low', 'high']]
df['time'] = pd.to_datetime(df['time'], unit='s')
df['slow_sma'] = df['close'].rolling(ssma_period).mean()
df['fast_sma'] = df['close'].rolling(fsma_period).mean()
df

In [68]:
bars = mt5.copy_rates_range(symbol, timeframe, current_day, one_year_before)
df = pd.DataFrame(bars)
df


""


In [86]:
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)
df = pd.DataFrame(bars)[['time', 'open', 'close', 'low', 'high']]
df['time'] = pd.to_datetime(df['time'], unit='s')
df['slow_sma'] = df['close'].rolling(ssma_period).mean()
df['fast_sma'] = df['close'].rolling(fsma_period).mean()

df.dropna(inplace=True)
#df
grouped = df.groupby([df['time'].dt.year, df['time'].dt.month])
monthly_data = {}

for (year, month), group in grouped:
    print(group, '---')
    monthly_key = f"{year}-{month:02}"
    monthly_data[monthly_key] = group

monthly_data

                   time     open    close      low     high    slow_sma  \
27  2023-10-10 20:50:00  148.700  148.710  148.689  148.725  148.673643   
28  2023-10-10 20:55:00  148.711  148.725  148.709  148.734  148.677464   
29  2023-10-10 21:00:00  148.724  148.723  148.698  148.727  148.682857   
30  2023-10-10 21:05:00  148.726  148.737  148.726  148.758  148.688464   
31  2023-10-10 21:10:00  148.741  148.732  148.712  148.744  148.694643   
..                  ...      ...      ...      ...      ...         ...   
145 2023-10-11 06:40:00  148.867  148.850  148.849  148.877  148.773571   
146 2023-10-11 06:45:00  148.851  148.859  148.851  148.877  148.779643   
147 2023-10-11 06:50:00  148.860  148.878  148.844  148.892  148.784857   
148 2023-10-11 06:55:00  148.878  148.871  148.871  148.887  148.790500   
149 2023-10-11 07:00:00  148.868  148.855  148.852  148.871  148.795929   

       fast_sma  
27   148.724857  
28   148.725929  
29   148.724786  
30   148.722929  
31   148.

{'2023-10':                    time     open    close      low     high    slow_sma  \
 27  2023-10-10 20:50:00  148.700  148.710  148.689  148.725  148.673643   
 28  2023-10-10 20:55:00  148.711  148.725  148.709  148.734  148.677464   
 29  2023-10-10 21:00:00  148.724  148.723  148.698  148.727  148.682857   
 30  2023-10-10 21:05:00  148.726  148.737  148.726  148.758  148.688464   
 31  2023-10-10 21:10:00  148.741  148.732  148.712  148.744  148.694643   
 ..                  ...      ...      ...      ...      ...         ...   
 145 2023-10-11 06:40:00  148.867  148.850  148.849  148.877  148.773571   
 146 2023-10-11 06:45:00  148.851  148.859  148.851  148.877  148.779643   
 147 2023-10-11 06:50:00  148.860  148.878  148.844  148.892  148.784857   
 148 2023-10-11 06:55:00  148.878  148.871  148.871  148.887  148.790500   
 149 2023-10-11 07:00:00  148.868  148.855  148.852  148.871  148.795929   
 
        fast_sma  
 27   148.724857  
 28   148.725929  
 29   148.724786  

In [14]:
# Finding Crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)


def find_crossovers(slow_sma, fast_sma, prev_fast_sma):
    if slow_sma < fast_sma and prev_fast_sma < slow_sma:
        return 'bull'
    elif slow_sma > fast_sma and prev_fast_sma > slow_sma:
        return 'bear'
    else:
        return None


df['crossover'] = np.vectorize(find_crossovers)(
    df['slow_sma'], df['fast_sma'], df['prev_fast_sma'])

signal = df[df['crossover'] == 'bull'].copy()
signal

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: invalid value encountered in find_crossovers (vectorized)
  outputs = ufunc(*inputs)


,time,open,close,low,high,slow_sma,fast_sma,prev_fast_sma,crossover
30,2023-10-09 18:10:00,148.741,148.764,148.734,148.766,148.648821,148.658786,148.644000,bull
89,2023-10-09 23:05:00,148.500,148.505,148.490,148.520,148.497536,148.501214,148.496500,bull
133,2023-10-10 02:45:00,148.577,148.622,148.574,148.662,148.411286,148.414500,148.396143,bull
134,2023-10-10 02:50:00,148.616,148.542,148.511,148.633,148.416821,148.430143,148.414500,bull


In [46]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)

fig.show()

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [10]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'

    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
            else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'

    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }


class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df

    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df

    def add_position(self, position):
        self.positions.append(position)

        return True

# logic
    def run(self):
        for i, data in self.data.iterrows():

            if data.crossover == 'bear':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)

            if data.crossover == 'bull':
                self.add_position(
                    Position(data.time, data.close, 'buy', self.volume, 0, 0))

        return self.get_positions_df()

In [15]:
sma_crossover_strategy = Strategy(df, 5000, 5)
result = sma_crossover_strategy.run()

result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2023-10-09 18:10:00,148.764,buy,5,0,0,2023-10-09 19:40:00,148.484,-1.400,closed,4998.600
1,2023-10-09 23:05:00,148.505,buy,5,0,0,2023-10-10 00:00:00,148.430,-0.375,closed,4998.225
2,2023-10-10 02:45:00,148.622,buy,5,0,0,NaT,NaN,NaN,open,NaN
3,2023-10-10 02:50:00,148.542,buy,5,0,0,NaT,NaN,NaN,open,NaN


In [48]:
px.line(result, x='close_datetime', y='pnl')

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [16]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)

for i, row in result[result['status'] == 'closed'].iterrows():

    if row.profit > 0:
        fig.add_shape(type="line",
                      x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
                      line=dict(color="Green", width=3)
                      )

    elif row.profit < 0:
        fig.add_shape(type="line",
                      x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
                      line=dict(color="Red", width=3)
                      )


fig.show()

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
while True:
    latest_bars = mt5.copy_rates_from_pos(symbol, timeframe, 0, ssma_period)
    df = pd.DataFrame(latest_bars)[['time', 'open', 'close', 'low', 'high']]
    
    df['slow_sma'] = df['close'].rolling(ssma_period).mean()
    df['fast_sma'] = df['close'].rolling(fsma_period).mean()
    df.dropna(inplace=True)

    df['prev_fast_sma'] = df['fast_sma'].shift(1)
    df['crossover'] = np.vectorize(find_crossovers)(df['slow_sma'], df['fast_sma'], df['prev_fast_sma'])

    df

    latest_crossover = df['crossover'].iloc[-1]

    condition_to_buy = latest_crossover == 'bull'

    condition_to_sell = latest_crossover == 'bear'

    if condition_to_buy:
        order = mt5.order_send(
            symbol=symbol,
            action=mt5.ORDER_BUY,
            volume=volume,
            price=mt5.symbol_info_tick(symbol).ask,
            deviation=20,
            type=mt5.ORDER_MARKET
        )

        if order.retcode != mt5.TRADE_RETCODE_DONE:
            print("Erro ao enviar ordem:", mt5.last_error())

    if condition_to_sell:
        position = mt5.positions_get(symbol=symbol)[0]
        order = mt5.order_send(
            symbol=symbol,
            action=mt5.ORDER_SELL,
            volume=volume,
            price=mt5.symbol_info_tick(symbol).bid,
            deviation=20,
            position=position.ticket,
            type=mt5.ORDER_MARKET
        )

        if order.retcode != mt5.TRADE_RETCODE_DONE:
            print("Erro ao fechar posição:", mt5.last_error())

    time.sleep(60) 

In [109]:
lot = 0.1
point = mt5.symbol_info(symbol).point
price = mt5.symbol_info_tick(symbol).ask
symbol_info = mt5.symbol_info(symbol)
deviation = 20
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": volume,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "sl": price - 100 * point,
    "tp": price + 100 * point,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
}
 

order = mt5.order_send(request)

print(order, 'order')

if order.retcode != mt5.TRADE_RETCODE_DONE:
    print(f"Erro ao enviar ordem. Código de retorno: {order.retcode}, Resultado: {order.comment}, Mensagem de Erro: {mt5.last_error()}")


OrderSendResult(retcode=10009, deal=319872549, order=379441897, volume=0.1, price=149.105, bid=149.099, ask=149.105, comment='Request executed', request_id=1890370004, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='USDJPY', volume=0.1, price=149.105, stoplimit=0.0, sl=149.005, tp=149.20499999999998, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0)) order


In [110]:
mt5.orders_get()

()